# Process OGC Web Feature Service

This notebook uses an endpoint URL, passed either as an 'endpoint' parameter when the notebook is called, or assigned in cell 5.



In [82]:
import sys
try:
    from osgeo import ogr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')
    
from ipywidgets import interact

import requests
import json
from pandas.io.json import json_normalize
maxfeatures=500000

#import numpy as np
#import pandas as pd
#import xmltodict
#from ipywidgets import interactive, fixed, interact_manual
#import ipywidgets as widgets

#for some reason the env variable is not getting set when the book is first opened, have to set manually
#WFS driver doesn't work
set GDAL_DATA=E:\EPrograms\Anaconda\envs\py36\Library\share\gdal

In [83]:
%%javascript
//check if there is an endpoint parameter in the notebook call

function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("endpoint='".concat(getQueryStringValue("endpoint")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [84]:
def the_envelope(geom):
    (minX, maxX, minY, maxY) = geom.GetEnvelope()
    # calculate a bounding box geometry for the given geometry argument
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(minX, minY)
    ring.AddPoint(maxX, minY)
    ring.AddPoint(maxX, maxY)
    ring.AddPoint(minX, maxY)
    ring.AddPoint(minX, minY)

    # Create polygon
    poly_envelope = ogr.Geometry(ogr.wkbPolygon)
    poly_envelope.AddGeometry(ring)
    return poly_envelope

In [85]:
def testurl(theurl):
    #try HEAD first in case the response document is big
    r = requests.head(theurl)
    if (r.status_code != requests.codes.ok):
        #check GET in case is an incomplete http implementation
        r = requests.get(theurl)
        if (r.status_code == requests.codes.ok):
            return True
        else:
            return False
    else:
        return True

In [86]:
url_partitioned = full_notebook_url.partition('ISOmetadata-ExtractDistributions.ipynb')
base_url = url_partitioned[0];

if(endpoint is not None and len(endpoint)>0):
    print('Got endpoint parameter:',endpoint)
else: 
    # some USGS ScienceBase Geoserver WFS endpoints:
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c5fce4b0aa151574a8ed'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/4f4e49cfe4b07f02db5da90e'

    # some NGDS end points
    endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/CAaqWellChemistry/MapServer/WFSServer'
    #endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/AZActiveFaults/MapServer/WFSServer'

    #EarthChem
    #endpoint='http://ecp.iedadata.org/wfs/ows?service=wfs'

    #smu thermal conductivity, geoserver
    #endpoint='http://geothermal.smu.edu:9000/geoserver/aasg-thermalconductivity/ows'

    #NOAA Watch/warning polygons
    #endpoint = 'https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/WFSServer'
    print('Assign endpoint:', endpoint)
# limit the number of features that will be processed


('Got endpoint parameter:', 'https://www.sciencebase.gov/catalogMaps/mapping/ows/514753a5e4b0694ee75b548a')


In [87]:
## Shapefile available?
driverName = "WFS"
# Set the driver (optional)
wfs_drv = ogr.GetDriverByName(driverName)

if wfs_drv is None:
    print ("%s driver not available.\n" % driverName)
else:
    print  ("%s driver IS available.\n" % driverName)

WFS driver IS available.



In [88]:
# !set GDAL_DATA=E:\EPrograms\Anaconda\Library\share\gdal
# !set GDAL_DATA

In [89]:
# Speeds up querying WFS capabilities for services with alot of layers
gdal.SetConfigOption('OGR_WFS_LOAD_MULTIPLE_LAYER_DEFN', 'NO')

# Set config for paging. Works on WFS 2.0 services and WFS 1.0 and 1.1 with some other services.
gdal.SetConfigOption('OGR_WFS_PAGING_ALLOWED', 'YES')
gdal.SetConfigOption('OGR_WFS_PAGE_SIZE', '10000')


#test that service is up with getcapabilities request

capurl=endpoint + '?service=wfs&request=getCapabilities'

if testurl(capurl):
    # Open the webservice
    wfs_ds = wfs_drv.Open('WFS:' + endpoint)
    #wfs_ds = wfs_drv.Open('http://ecp.iedadata.org/wfs/ows?service=wfs')
    if wfs_ds is None:
        print('ERROR: can not open WFS datasource')
else:
    print('ERROR: service not responding')


In [90]:
# iterate over available layers
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    srs = layer.GetSpatialRef()
    print ('Layer: %s, Features: %s, SR: %s...' % (layer.GetName(), layer.GetFeatureCount(), srs.ExportToWkt()[0:50]))

    # iterate over features
    #fcount = 1
    #feat = layer.GetNextFeature()
    #while feat is not None:
    #    feat = layer.GetNextFeature()
    #    fcount = fcount + 1
        # do something more..
    #feat = None
    print ('feature count: ',layer.GetFeatureCount())

Layer: sb:footprint, Features: 1, SR: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84"...
('feature count: ', 1)
Layer: sb:pgv, Features: 4, SR: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84"...
('feature count: ', 4)


#gdal call to get the capabilities document
caps=wfs_ds.GetLayerByName('WFSGetCapabilities')
#print the number of feature types reported in the capabilities document
print(caps.GetFeatureCount())

for feat in caps:
    jsonwrap= json.loads(feat.ExportToJson())
    
capsxml=jsonwrap['properties']['content']
capsjson=xmltodict.parse(capsxml)
capsjson=json.dumps(capsjson, indent=2)
#print(capsjson)

Want to let user pick the layer to work with.

In [91]:

feat_list={}
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    feat_list[layer.GetName()] = layer.GetName()


def f(feat_menu):
    return feat_menu
# out = interact(f, feat_menu=nb_menu);
out = interact(f, feat_menu=feat_list.keys());

#print("interact out: ", out)

interactive(children=(Dropdown(description=u'feat_menu', options=('sb:pgv', 'sb:footprint'), value='sb:pgv'), â€¦

In [92]:
# Get a specific layer

chosenlayer= feat_list[out.widget.result]
if len(chosenlayer)==0:
    chosenlayer=wfs_ds.GetLayerByIndex(1).GetName()

layer = wfs_ds.GetLayerByName(chosenlayer)

print('chosen layer:',chosenlayer)

if not layer:
    sys.exit('ERROR: can not find layer in service')
else:
    pass

('chosen layer:', 'sb:pgv')


In [93]:
layer.GetFeatureCount()

4

In [94]:
%%time
count = 0
tdata = []

if (layer.GetFeatureCount()>0):
    feat=layer.GetFeature(1)
    print(feat)
#    jtemp=feat.GetGeometryRef().ExportToJson()
#    print('J temp:',jtemp)
#    tdata.append(json.loads(jtemp))   
#else:
#feat = layer.GetNextFeature()

#while feat is not None:
    for feat in layer:
        print('feature: %s' % feat.geometry().GetGeometryName())
        #ExportToJson doesn't work with some geometries (e.g .MULTISURFACE(??), 
        # in that case, replace with bounding box     
        try:
            temp=feat.ExportToJson()
        except:
            msenvelope=the_envelope( feat.geometry())
            #print('BoundingBox: %s' % msenvelope)
            print('replace geometry with bounding box')
            feat.SetGeometry(msenvelope)
        tdata.append(json.loads(feat.ExportToJson()))
        #feat = layer.GetNextFeature()
        count = count + 1
        #avoid getting hung on very long responses, maxsfeatures set in 
        if count > maxfeatures:
            break
feat = None

#print (tdata)
dataframe=json_normalize(tdata)
print('size:',dataframe.size)

#dataframe
print('done timed cell, %s features in result set' % count)

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x0000000011B11FC0> >
feature: MULTISURFACE
replace geometry with bounding box
feature: MULTISURFACE
replace geometry with bounding box
feature: MULTISURFACE
replace geometry with bounding box
feature: MULTISURFACE
replace geometry with bounding box
('size:', 44)
done timed cell, 4 features in result set
Wall time: 2.15 s


Problem with json_normalize:  
This appears to be a bug in the latest version of pandas: https://github.com/pandas-dev/pandas/issues/21158
Running pandas '0.23.0', error arises due to condition case when null value occurs on the nesting level greater than 0. It seems to have been changed around two months ago that seems to have made it's way into 0.23.0 release two weeks ago:

https://github.com/pandas-dev/pandas/commit/01882ba5b4c21b0caf2e6b9279fb01967aa5d650#diff-9c654764f5f21c8e9d58d9ebf14de86d

When I use python verion 3.6.3 :: Anaconda Inc. and pandas version 0.20.3 I do not see this issue and json_normalize is able to work properly. 

2018-06-28 SMR rolled pandas back to 0.20.3 in the py36 kernel env.


Now get a list of the attributes of the feature type

    #example function to get field definitions from gdal
    #these are more informative than the pandas dataframe definitions
    lyrDefn = layer.GetLayerDefn()
    for i in range( lyrDefn.GetFieldCount() ):
        fieldName =  lyrDefn.GetFieldDefn(i).GetName()
        fieldTypeCode = lyrDefn.GetFieldDefn(i).GetType()
        fieldType = lyrDefn.GetFieldDefn(i).GetFieldTypeName(fieldTypeCode)
        fieldWidth = lyrDefn.GetFieldDefn(i).GetWidth()
        GetPrecision = lyrDefn.GetFieldDefn(i).GetPrecision()

        print (fieldName + " - " + fieldType+ " " + str(fieldWidth) + " " + str(GetPrecision))

Do some data summarization with pandas

#easy pandas way to get the field types
#dataframe.dtypes

In [95]:
#for value in dataframe['properties.K_mgL']:
#    if value != None:
#        print (value)
lyrDefn = layer.GetLayerDefn()

for col in dataframe.columns:
    if (col.find('.')>-1):
        field=str(col.split('.')[1])
        #print('col: %s, field: %s' % (col, field))
        fielddef = lyrDefn.GetFieldDefn(lyrDefn.GetFieldIndex(field))
        if (fielddef is not None):
            fieldTypeCode = fielddef.GetType()
            fieldType = fielddef.GetFieldTypeName(fieldTypeCode)
            fieldWidth = fielddef.GetWidth()
            GetPrecision = fielddef.GetPrecision()       
            print (field + " - " + fieldType+ " " + str(fieldWidth) + " " + str(GetPrecision))
        else:
            print(field + ' has no ogr definition')

            
def findfieldtype(pcol):
    if (pcol.find('.')>-1):
        field=str(pcol.split('.')[1])
    else:
        field=str(pcol)
    fielddef = lyrDefn.GetFieldDefn(lyrDefn.GetFieldIndex(field))
    if (fielddef is not None):
        thetype= fielddef.GetFieldTypeName(fielddef.GetType())
    else:
        thetype='xxxx'
    return thetype

coordinates has no ogr definition
type has no ogr definition
AREA - Real 0 0
GRID_CODE - Integer64 0 0
PERIMETER - Real 0 0
PGAPOL_ - Integer64 0 0
PGAPOL_ID - Integer64 0 0
VALUE - Real 0 0
gml_id - String 0 0


In [77]:
print('%s features analyzed.' % len(dataframe.index))

for col in dataframe.columns:
    #print(col)
    if (col.find('.')>-1):
        colname=str(col.split('.')[1])
    else:
        colname=str(col)
        
    if (dataframe[col].count() > 0) and  (dataframe[col].dtype=='float64'):
        mean=dataframe[col].mean()
        max=dataframe[col].max()
        min=dataframe[col].min()
        print('field:%s, count:%s, mean:%s; max:%s; min:%s' 
              % (colname, dataframe[col].count(), "{:.2f}".format(mean), 
                 "{:.2f}".format(max), "{:.2f}".format(min)))
    
    if (dataframe[col].count() > 0) and  (findfieldtype(col).find('Integer')>-1):
        max=dataframe[col].max()
        min=dataframe[col].min()
        print('field:%s, count:%s: max:%s; min:%s' 
              % (colname, dataframe[col].count(),  
                 "{:.2f}".format(max), "{:.2f}".format(min)))
    
    if (dataframe[col].count() > 0) and  (findfieldtype(col)=='String'):
        # and  (type(dataframe[col][1])=='str')
        #for item in dataframe[col].unique():
        #    print (item)
        #print (type(dataframe[col][1]))
        if len(dataframe.index)==len(dataframe[col].unique()):
            print(colname + ' is a text field, all values are unique')
            print('  ')
        elif (len(dataframe[col].unique()) > 50):
            print(colname + ' appears to be a free text field, there are ' + str(len(dataframe[col].unique())) + ' unique values.' )
            print('  ')
        else:
            print (colname +' has '+ str(len(dataframe[col].unique())) + ' unique values.')
            for val in dataframe[col].unique(): print(str(val))
            print('  ')
 
emptyfields = []
print('\nThe following fields contain no data:')
for col in dataframe.columns:            
    if (dataframe[col].count() == 0):
        emptyfields.append(str(col.split('.')[1]))       
print(emptyfields)
        

4 features analyzed.
field:AREA, count:4, mean:0.00; max:0.00; min:0.00
field:GRID_CODE, count:4: max:1.00; min:1.00
field:PERIMETER, count:4, mean:0.00; max:0.00; min:0.00
field:PGAPOL_, count:4: max:5.00; min:2.00
field:PGAPOL_ID, count:4: max:4.00; min:1.00
field:VALUE, count:4, mean:1.00; max:1.00; min:1.00
gml_id is a text field, all values are unique
  

The following fields contain no data:
[]


In [96]:
 dataframe.describe()


,id,properties.AREA,properties.GRID_CODE,properties.PERIMETER,properties.PGAPOL_,properties.PGAPOL_ID,properties.VALUE
count,4.000000,4.0,4.0,4.0,4.000000,4.000000,4.0
mean,2.500000,0.0,1.0,0.0,3.500000,2.500000,1.0
std,1.290994,0.0,0.0,0.0,1.290994,1.290994,0.0
min,1.000000,0.0,1.0,0.0,2.000000,1.000000,1.0
25%,1.750000,0.0,1.0,0.0,2.750000,1.750000,1.0
50%,2.500000,0.0,1.0,0.0,3.500000,2.500000,1.0
75%,3.250000,0.0,1.0,0.0,4.250000,3.250000,1.0
max,4.000000,0.0,1.0,0.0,5.000000,4.000000,1.0
